In [13]:
from webScraper import *
import pandas as pd
import csv
from openpyxl import load_workbook

<font size="5">
    Step 1: Scrape Latest Articles
</font>

In [2]:
# Scrape latest articles from https://nos.nl
# Outputs latest articles in 'articles_scrapped.json'
webDataScrape()

<font size="5">
    Step 2: Add to Master Article List
</font>

In [3]:
# Updates master_article_list.json, using articles_scrapped.json as input
updateMasterList()

In [4]:
# Check length of master article list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))
print(len(master_article_list))

297


<font size="5">
    Step 3: Convert text in Articles to list of words with frequency
</font>

In [5]:
# Retrieve all articles from master list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))

In [6]:
# Retrieve list of words from each article
full_word_list = []
for article in master_article_list:
    full_word_list.extend(article.retrieveWordList())
print(full_word_list)

['IN', 'BRUSSEL', 'HEEFT', 'DE', 'POLITIE', 'INGEGREPEN', 'OP', 'DE', 'EERSTE', 'DAG', 'VAN', 'DE', 'NATIONAL', 'CONSERVATISM', 'CONFERENCE', 'NATCON', "ZO'N", 'VEERTIG', 'CONSERVATIEVE', 'EN', 'RECHTSE', 'POLITICI', 'ZOUDEN', 'SPREKEN', 'OP', 'DIE', 'CONFERENTIE', 'ONDER', 'WIE', 'DE', 'BRITSE', 'MINISTER', 'SUELLA', 'BRAVERMAN', 'EN', 'DE', 'FRANSE', 'POLITICUS', 'ÉRIC', 'ZEMMOUR', 'BURGEMEESTER', 'VAN', 'DE', 'BRUSSELSE', 'DEELGEMEENTE', 'SINT-JOOST-TEN-NODE', 'EMIR', 'KIR', 'VERBOOD', 'HET', 'EVENEMENT', 'KORT', 'NA', 'DE', 'START', 'UIT', 'VREES', 'VOOR', 'DE', 'VERSTORING', 'VAN', 'DE', 'OPENBARE', 'ORDE', 'DE', 'POLITIE', 'GREEP', 'IN', 'TOEN', 'DE', 'BRITSE', 'POLITICUS', 'NIGEL', 'FARAGE', 'OP', 'HET', 'PODIUM', 'STOND', 'DE', 'GEESTELIJK', 'VADER', 'VAN', 'DE', 'BREXIT', 'DE', 'ZAAL', 'WERD', 'NIET', 'ONTRUIMD', 'MAAR', 'DE', 'POLITIE', 'LIET', 'GEEN', 'NIEUWE', 'BEZOEKERS', 'MEER', 'TOE', 'BEZOEKERS', 'DIE', 'NAAR', 'BUITEN', 'GINGEN', 'KONDEN', 'DAARNA', 'DE', 'ZAAL', 'NIET

In [7]:
# Converts word list to dictionary containing count of each word
word_list_count_dict = {}
for word in full_word_list:
    if word in word_list_count_dict.keys():
        word_list_count_dict[word] = word_list_count_dict[word] + 1
    else:
        word_list_count_dict[word] = 1
print(word_list_count_dict)

{'IN': 2700, 'BRUSSEL': 25, 'HEEFT': 449, 'DE': 7275, 'POLITIE': 179, 'INGEGREPEN': 6, 'OP': 1357, 'EERSTE': 114, 'DAG': 68, 'VAN': 3302, 'NATIONAL': 2, 'CONSERVATISM': 1, 'CONFERENCE': 4, 'NATCON': 1, "ZO'N": 106, 'VEERTIG': 4, 'CONSERVATIEVE': 11, 'EN': 2221, 'RECHTSE': 5, 'POLITICI': 24, 'ZOUDEN': 71, 'SPREKEN': 15, 'DIE': 1042, 'CONFERENTIE': 11, 'ONDER': 206, 'WIE': 43, 'BRITSE': 17, 'MINISTER': 75, 'SUELLA': 1, 'BRAVERMAN': 1, 'FRANSE': 19, 'POLITICUS': 9, 'ÉRIC': 1, 'ZEMMOUR': 1, 'BURGEMEESTER': 39, 'BRUSSELSE': 5, 'DEELGEMEENTE': 2, 'SINT-JOOST-TEN-NODE': 2, 'EMIR': 1, 'KIR': 1, 'VERBOOD': 3, 'HET': 3440, 'EVENEMENT': 7, 'KORT': 39, 'NA': 188, 'START': 13, 'UIT': 385, 'VREES': 7, 'VOOR': 1015, 'VERSTORING': 3, 'OPENBARE': 15, 'ORDE': 27, 'GREEP': 5, 'TOEN': 118, 'NIGEL': 1, 'FARAGE': 1, 'PODIUM': 5, 'STOND': 22, 'GEESTELIJK': 2, 'VADER': 15, 'BREXIT': 2, 'ZAAL': 7, 'WERD': 288, 'NIET': 795, 'ONTRUIMD': 6, 'MAAR': 565, 'LIET': 25, 'GEEN': 252, 'NIEUWE': 143, 'BEZOEKERS': 17, 'ME

In [8]:
# Convert to Dataframe and sort
word_count_df = pd.DataFrame(columns=['word', 'count'])
for word, count in word_list_count_dict.items():
    word_count_df.loc[len(word_count_df.index)] = [word, count]

word_count_sorted_df = word_count_df.sort_values(by=['count'], ascending=False)

In [9]:
# Output sorted word list to csv
word_count_sorted_df.to_csv('word_list.csv', index=False)

<font size="5">
    Step 4: Remove Invalid Words and Custom Filtered Words
</font>

In [10]:
# Remove words from invalid_words.csv - These are either symbol characters, names of people etc.
with open('invalid_words.csv', newline='') as csv_file:
    reader = csv.reader(csv_file)
    invalid_word_list = [element[0] for element in list(reader)]

# Remove already learned words
with open('learned_words.csv', newline='') as csv_file:
    reader = csv.reader(csv_file)
    learned_words_list = [element[0] for element in list(reader)]

# Combine lists
words_to_remove_list = invalid_word_list + learned_words_list

In [11]:
# Remove words from current dataframe word count list
word_count_filtered_df = word_count_sorted_df.copy()
for remove_word in words_to_remove_list:
    word_count_filtered_df = word_count_filtered_df[word_count_filtered_df.word !=remove_word]


In [32]:

top_1000_words_df = word_count_filtered_df.sort_values('count', ascending=False).head(1000).reset_index(drop=True)
top_1000_words_df = top_1000_words_df.drop(['count'], axis=1)

In [34]:
# Output filtered word list to excel
#word_count_filtered_df.to_csv('word_list_filtered.csv', index=False)
wb = load_workbook(filename='text_translator.xlsm', keep_vba=True)
ws = wb['Sheet1']

for index, row in top_1000_words_df.iterrows():
    cell = 'A%d' % (index + 2)
    ws[cell] = row[0]

wb.save('text_translator.xlsm')

C:\Users\mikey\AppData\Local\Temp\ipykernel_10096\2312205465.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ws[cell] = row[0]
